In [ ]:
!pip install beautifulsoup4
!pip install requests
!pip install fake_useragent
!pip install pandas numpy

In [1]:
import requests      
import numpy as np   
import pandas as pd          
from bs4 import BeautifulSoup
from fake_useragent import UserAgent

from time import sleep

import json
import os
import glob

In [2]:
user_agent = UserAgent().firefox

In [3]:
base_url = 'https://www.gastronom.ru'
catalog_url = 'https://www.gastronom.ru/recipe' # Каталог рецептов

In [26]:
res = requests.get(catalog_url, headers={'User-Agent': user_agent})
bs = BeautifulSoup(res.content, 'html.parser')

In [27]:
urls = bs.find_all('a', attrs={'class': '_subtopic_1288r_9'})
urls = [(i['href'], i.text) for i in urls]
# urls

In [28]:
catalog = pd.DataFrame(urls, columns=['url', 'title'])
catalog

,url,title
0,/group/recepty-iz-kapusty-3644,Рецепты из капусты
1,/group/blyuda-iz-kartofelya-1469,Блюда с картошкой
2,/group/recepty-s-gribami-1590,Рецепты с грибами
3,/group/pirogi-s-yablokami-yablochnyj-pirog-1837,Яблочные пироги
4,/group/klyukva-1964,Рецепты с клюквой
...,...,...
67,/group/morsy-1051,Морсы
68,/group/kofe-1484,Кофе
69,/group/domashnij-limonad-1661,Лимонад
70,/group/kompoty-1348,Компоты


In [97]:
url = base_url+catalog.url[0]+'?page=8'
res = requests.get(url, headers={'User-Agent': user_agent})
bs = BeautifulSoup(res.content, 'html.parser')

In [84]:
urls = bs.find_all('a', attrs={'class': '_link_1ll11_14'})
urls = [i['href'] for i in urls]

In [98]:
last_page_number = bs.find_all('a', attrs={'class': '_root_41sm3_2'})
last_page_number[-2].text

'27'

In [86]:
base_url+urls[0]
urls, len(urls)

(['/recipe/47161/yablochnyj-pirog',
  '/recipe/21333/sharlotka-nezhnaya',
  '/recipe/18999/pirog-neobyknovennyj',
  '/recipe/14695/yablochnyj-shtrudel',
  '/recipe/44842/ovsyanyj-pirog-s-tykvoj-i-yablokami',
  '/recipe/12395/sharlotka-parizhskaya',
  '/recipe/11853/otkrytyj-yablochnyj-pirog',
  '/recipe/11623/yablochnyj-shtrudel-aromaty-zimy',
  '/recipe/11366/yablochnyj-pirog-sharlotka',
  '/recipe/50102/shtrudel-s-yablokami-i-grushami',
  '/recipe/10490/sharlotka',
  '/recipe/10470/yablochnyj-pirog-sharlotka',
  '/recipe/8505/pirog-so-slivami',
  '/recipe/6753/aronichnyj-pirog-s-chernoplodkoj',
  '/recipe/6438/yablochnyj-pirog',
  '/recipe/5571/yablochnyj-pirog',
  '/group/pirogi-recepty-1060',
  '/group/recepty-iz-yablok-1834',
  '/group/vypechka-s-yablokami-1836'],
 19)

# Выбираем данные со страницы

In [53]:
url = 'https://www.gastronom.ru/recipe/30233/prostaja-sharlotka-s-jablokami'
res = requests.get(url, headers={'User-Agent': user_agent})
bs = BeautifulSoup(res.content, 'html.parser')

In [54]:
bs.find_all('h1', attrs={'class': '_root_ofawm_2 _title_ucjtl_46'})[0].text

'Простая шарлотка с яблоками'

In [55]:
bs.find_all('span', attrs={'data-testid': 'rating'})[0].text

'4.58'

In [57]:
bs.find_all('span', attrs={'class': '_date_1yfux_29'})[0].text

'19 августа 2022 г.'

In [59]:
# bs.find_all('span', attrs={'class': '_bold_1prcq_17'})[0].text

'легко'

In [65]:
[i.text for i in bs.find_all('span', attrs={'class': '_bold_1prcq_17'})]

['легко', '148.91 ккал/порция ', '8', ' 50 мин', '3.83 г/2.37 г/27.62']

In [64]:
[i.text for i in bs.find_all('span', attrs={'class': '_tag_1178o_9'})]

['Духовка', 'Выпекание', 'Выпечка', 'Вечеринка, фуршет', 'Детский праздник']

In [76]:
'\n'.join([i.text for i in bs.find_all('div', attrs={'class': '_editorjsContent_1x6jq_2'})]).strip()

'Простая шарлотка с яблоками — пирог, с которого многие начинают осваивать выпечку. И этот тот самый случай, когда начинающий кулинар легко сможет поверить в свои силы! Шарлотка готовится необыкновенно легко (особенно, если взбивать тесто миксером, как и рекомендуется в рецепте), а получается неизменно вкусной и нравится практически всем. Вообще это блюдо — явление удивительное! Сколько бы вы ни осваивали рецептов яблочных пирогов, даже самых затейливых, близкие нет-нет, да и попросят вас испечь простую шарлотку с яблоками. Потому что она — родная, понятная и необыкновенно вкусная!\nШаг 1\n\n\nПриготовьте тесто для простой шарлотки с яблоками. Яйца вбейте в чашу блендера. Взбивайте до получения пышной светлой массы. Положите соль и перемешайте.\n\nШаг 2\n\n\nПостепенно, взбивая миксером на высокой скорости, всыпьте весь сахар. Взбивайте до тех пор, пока в чаше не образуется пышная масса однородной консистенции.\nШаг 3\n\n\nНа яичную массу просейте через частое сито муку. Тщательно пере

In [67]:
[i.text for i in bs.find_all('div', attrs={'itemprop': 'recipeIngredient'})]

['яйца – 3 шт.',
 'сахар – 110–120 г',
 'соль – на кончике ножа',
 'мука – 100 г',
 'яблоки – 3-4 шт.']

In [4]:
def get_info(url):
    res = requests.get(url, headers={'User-Agent': user_agent})
    bs = BeautifulSoup(res.content, 'html.parser')

    recipe_desc = [i.text for i in bs.find_all('span', attrs={'class': '_bold_1prcq_17'})]
    
    info = {
        'title': bs.find_all('h1', attrs={'class': '_root_ofawm_2 _title_ucjtl_46'})[0].text,
        'rating': bs.find_all('span', attrs={'data-testid': 'rating'})[0].text, 
        'date': bs.find_all('span', attrs={'class': '_date_1yfux_29'})[0].text, 
        'complexity': recipe_desc[0] if len(recipe_desc) >= 1 else '',
        'calories': recipe_desc[1] if len(recipe_desc) >= 2 else '',
        'number_of_servings': recipe_desc[2] if len(recipe_desc) >= 3 else '',
        'time': recipe_desc[3] if len(recipe_desc) >= 4 else '',
        'proteins fats carbohydrates': recipe_desc[4] if len(recipe_desc) >= 5 else '',
        'tags': [i.text for i in bs.find_all('span', attrs={'class': '_tag_1178o_9'})],
        'text': '\n'.join([i.text for i in bs.find_all('div', attrs={'class': '_editorjsContent_1x6jq_2'})]).strip(),
        'ingredients': [i.text for i in bs.find_all('div', attrs={'itemprop': 'recipeIngredient'})],
    }
    
    
    return info

In [80]:
get_info('https://www.gastronom.ru/recipe/55021/jablochnyj-pirog-na-syrnom-teste')

{'title': 'Яблочный пирог на сырном тесте',
 'rating': '5.00',
 'date': '08 мая 2022 г.',
 'complexity': 'средне',
 'calories': '322.45 ккал/порция ',
 'number_of_servings': '10',
 'time': ' 2 ч',
 'proteins fats carbohydrates': '5.50 г/14.30 г/46.11',
 'tags': ['Выпечка'],
 'text': 'Яблочный пирог на сырном тесте весьма популярен в Северной Америке. За океаном его готовят на чеддере, имеющем приятный ореховый привкус. Его можно заменить гаудой или любым твердым сыром, который нравится лично вам. Из козьего молока тоже подойдет. Приверженцам традиционной гастрономии можем предложить компромиссную замену на творог. Но в этом случае вы не откроете для себя удивительные гастрономические сочетания, которые, уверены, станут вашими любимыми. К тому же сыр благотворно влияет на структуру выпечки, ее качество в готовом варианте – она получается не сдобной, а немного слоеной, рублено-песочной. А контрастная комбинация солоноватой основы и сладкой начинки дает необыкновенный вкус – любители пиро

# Парсим

In [5]:
def get_urls(url):
    res = requests.get(url, headers={'User-Agent': user_agent})
    bs = BeautifulSoup(res.content, 'html.parser')
    urls = bs.find_all('a', attrs={'class': '_link_1ll11_14'})
    urls = [i['href'] for i in urls]
    return urls

In [136]:
tmp = get_urls('https://www.gastronom.ru/group/recepty-iz-kapusty-3644')
len(tmp)

49

In [10]:
path = 'data/'
os.mkdir(path)

FileExistsError: [Errno 17] File exists: 'data/'

In [134]:
def save_urls(index, urls_list):
    with open(path+str(index)+'.txt', 'w') as f:
        f.write(json.dumps(urls_list))

In [138]:
save_urls(-99, all_urls)

In [157]:
quantity = 0

for index, row in catalog.iterrows():
    # print(row['url'], row['title'])
    url = base_url+row['url']
    res = requests.get(url, headers={'User-Agent': user_agent})
    bs = BeautifulSoup(res.content, 'html.parser')
    last_page_number = bs.find_all('a', attrs={'class': '_root_41sm3_2'})
    # print(last_page_number, url)
    amount = 1
    if len(last_page_number) > 0:
        amount = int(last_page_number[-2].text)
    
    all_urls = []
    print(row['title'])
    print(f'\tКоличество страниц с рецептами: {amount}')
    for i in range(1, amount+1):
        all_urls.extend(get_urls(f'{url}?page={i}'))
            
    save_urls(index, all_urls)
        
    quantity += len(all_urls)
    print(f'Количество собранных url: {quantity}')
    

Рецепты из капусты
	Количество страниц с рецептами: 27


KeyboardInterrupt: 

In [159]:
catalog[:11]

,url,title
0,/group/recepty-iz-kapusty-3644,Рецепты из капусты
1,/group/blyuda-iz-kartofelya-1469,Блюда с картошкой
2,/group/recepty-s-gribami-1590,Рецепты с грибами
3,/group/pirogi-s-yablokami-yablochnyj-pirog-1837,Яблочные пироги
4,/group/klyukva-1964,Рецепты с клюквой
5,/group/blyuda-iz-tykvy-1950,Блюда из тыквы
6,/group/prostye-salaty-2839,Простые салаты
7,/group/salat-olive-2116,Салат Оливье
8,/group/salat-cezar-1586,Салат Цезарь
9,/group/salat-nisuaz-1335,Салат Нисуаз


In [3]:
page_path = 'pages/'
# os.mkdir(page_path)

In [7]:
def load_urls(file_path):
    with open(file_path, 'r') as f:
        return json.loads(f.read())

In [4]:
def save_recipe(path, data):
    with open(path, 'w') as f:
        f.write(json.dumps(data))
        
def load_recipe(path):
    with open(path, 'r') as f:
        return json.loads(f.read())

In [ ]:
import glob

files = glob.glob(path+'*')

counter = 626

for i in files:
    recipes_url = load_urls(i)
    index = int(i[len(page_path)-1:-1*len('.txt')])

    for j in recipes_url:
        try:
            recipe = get_info(base_url+j)
            recipe['category'] = catalog['title'][index]
            save_recipe(f'{page_path}{counter}', recipe)
            print(f'Counter: {counter}')
            counter+=1
        except:
            print('Error')


Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error


In [185]:
int('pages/0.txt'[len(page_path):-1*len('.txt')])

0

In [171]:
catalog['title'][2]

'Рецепты с грибами'

In [ ]:
recipes = []

for i in range(counter):
    recipes.append()

In [190]:
f = glob.glob(path+'*')
for i in sorted(f):
    print(int(i[len(page_path)-1:-1*len('.txt')]))

0
1
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
3
30
31
32
33
34
35
36
37
38
39
4
40
41
42
43
44
45
46
47
5
6
7
70
71
9


ValueError: invalid literal for int() with base 10: 'z_2'

In [2]:
page_path='pages/'

In [5]:
json_list = [load_recipe(i) for i in glob.glob(page_path+'*')]

In [18]:
df =  pd.DataFrame(json_list)
df

,title,rating,date,complexity,calories,number_of_servings,time,proteins fats carbohydrates,tags,text,ingredients,category
0,Пицца Мясное ассорти,5.00,11 сентября 2023 г.,легко,585.02 ккал/порция,2,30 мин,30.61 г/36.17 г/34.09,"[Основное блюдо, Вегетарианство]","Если вы любите пиццу Пепперони, то вам обязате...","[основа для пиццы – 1 шт., томатный соус – 80 ...",Пицца
1,Котлеты из цветной капусты,0.00,18 ноября 2020 г.,средне,4,,,,"[Основное блюдо, Вегетарианство: Ово-лакто, Ве...","Котлеты из цветной капусты, которые так понрав...","[капуста цветная 1 кг, яйцо куриное - 2 шт., м...",Салат Цезарь
2,Картофельное пюре с чесноком и беконом,5.00,16 февраля 2015 г.,легко,459.22 ккал/порция,4,45 мин,15.38 г/30.66 г/30.20,"[Жарение, Обед, Ужин, Основное блюдо, Вегетари...","Шаг 1\n\n\nОчистите картофель, разрежьте кажду...","[1 кг картофеля для пюре, 250–300 г тонких лом...",Блюда с картошкой
3,Пирожки с картофелем и луком,5.00,11 декабря 2017 г.,средне,266.92 ккал/порция,15,2 ч 15 мин,1 ч,"[Хлебопечка, Выпекание, Жарение, Выпечка, Вече...","Румяные, мягкие домашние пирожки с картофелем ...","[мука пшеничная 600 г, дрожжи быстродействующи...",Блюда с картошкой
4,Блины закусочные Традиционные,0.00,30 сентября 2011 г.,легко,1,1 ч,,,"[Основное блюдо, Вегетарианство]","Шаг 1Взбить миксером яйца с сахаром, солью и м...","[семга слабосоленая – 150 г, сметана – 175-200...",Блины и блинчики
...,...,...,...,...,...,...,...,...,...,...,...,...
10500,Гречневые котлеты с фасолью и соусом из авокадо,0.00,23 марта 2014 г.,средне,1,1 ч 30 мин,,,"[Основное блюдо, Вегетарианство: Строго, Вегет...","Шаг 1\n\n\nПриготовить ""фарш"" для котлет. Зали...",[фасоль «Черный глаз» белая пестрая ТМ Мистр...,Котлеты
10501,Паштет из куриной печени с коньяком и гранатов...,4.33,14 декабря 2014 г.,средне,12,40 мин,15 мин,,"[Европейская кухня, Завтрак, Закуска, Вегетари...",Очень вкусный и нежный паштет из куриной печен...,"[куриная печень - 500 г, лук репчатый - 1 шт.,...",Праздничные закуски
10502,Капоната,4.50,22 июня 2007 г.,легко,183.38 ккал/порция,4,30 мин,2.82 г/8.97 г/22.41,"[Итальянская кухня, Закуска, Вегетарианство: С...","Капоната – сицилийское блюдо из баклажанов, по...","[2 стебля черешкового сельдерея, 100 г зеленых...",Тушеные овощи
10503,Сочень архангельский с рыбой,0.00,16 января 2008 г.,средне,948.00 ккал/порция,6,40.00 г/56.00 г/73.00,,[Выпечка],"Шаг 1Пшеничную и ржаную муку просеять в миску,...","[мука ржаная – 100 г, масло сливочное – 250 г,...",Пироги


In [24]:
df

,calories,carbohydrates,category,complexity,date,fats,ingredients,number_of_servings,proteins,proteins fats carbohydrates,rating,tags,text,time,title
0,585.02,34.09,Пицца,легко,2023-09-11,36.17,"[основа для пиццы – 1 шт., томатный соус – 80 ...",2,30.61,30.61 г/36.17 г/34.09,5.00,"[Основное блюдо, Вегетарианство]","Если вы любите пиццу Пепперони, то вам обязате...",30,Пицца Мясное ассорти
1,0.00,NaN,Салат Цезарь,средне,2020-11-18,NaN,"[капуста цветная 1 кг, яйцо куриное - 2 шт., м...",4,NaN,,0.00,"[Основное блюдо, Вегетарианство: Ово-лакто, Ве...","Котлеты из цветной капусты, которые так понрав...",0,Котлеты из цветной капусты
2,459.22,30.20,Блюда с картошкой,легко,2015-02-16,30.66,"[1 кг картофеля для пюре, 250–300 г тонких лом...",4,15.38,15.38 г/30.66 г/30.20,5.00,"[Жарение, Обед, Ужин, Основное блюдо, Вегетари...","Шаг 1\n\n\nОчистите картофель, разрежьте кажду...",45,Картофельное пюре с чесноком и беконом
3,266.92,NaN,Блюда с картошкой,средне,2017-12-11,NaN,"[мука пшеничная 600 г, дрожжи быстродействующи...",15,NaN,1 ч,5.00,"[Хлебопечка, Выпекание, Жарение, Выпечка, Вече...","Румяные, мягкие домашние пирожки с картофелем ...",135,Пирожки с картофелем и луком
4,0.00,NaN,Блины и блинчики,легко,2011-09-30,NaN,"[семга слабосоленая – 150 г, сметана – 175-200...",1,NaN,,0.00,"[Основное блюдо, Вегетарианство]","Шаг 1Взбить миксером яйца с сахаром, солью и м...",0,Блины закусочные Традиционные
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10500,0.00,NaN,Котлеты,средне,2014-03-23,NaN,[фасоль «Черный глаз» белая пестрая ТМ Мистр...,1,NaN,,0.00,"[Основное блюдо, Вегетарианство: Строго, Вегет...","Шаг 1\n\n\nПриготовить ""фарш"" для котлет. Зали...",0,Гречневые котлеты с фасолью и соусом из авокадо
10501,0.00,NaN,Праздничные закуски,средне,2014-12-14,NaN,"[куриная печень - 500 г, лук репчатый - 1 шт.,...",12,NaN,,4.33,"[Европейская кухня, Завтрак, Закуска, Вегетари...",Очень вкусный и нежный паштет из куриной печен...,15,Паштет из куриной печени с коньяком и гранатов...
10502,183.38,22.41,Тушеные овощи,легко,2007-06-22,8.97,"[2 стебля черешкового сельдерея, 100 г зеленых...",4,2.82,2.82 г/8.97 г/22.41,4.50,"[Итальянская кухня, Закуска, Вегетарианство: С...","Капоната – сицилийское блюдо из баклажанов, по...",30,Капоната
10503,948.00,73.00,Пироги,средне,2008-01-16,56.00,"[мука ржаная – 100 г, масло сливочное – 250 г,...",6,40.00,40.00 г/56.00 г/73.00,0.00,[Выпечка],"Шаг 1Пшеничную и ржаную муку просеять в миску,...",0,Сочень архангельский с рыбой


In [7]:
df.columns

Index(['title', 'rating', 'date', 'complexity', 'calories',
       'number_of_servings', 'time', 'proteins fats carbohydrates', 'tags',
       'text', 'ingredients', 'category'],
      dtype='object')

In [19]:
def to_min(data):
    minutes = data['time']
    
    if 'ч' in minutes:
        time = minutes.strip().replace('ч', '').split()
        
        data['time'] = int(time[0]) * 60 + (int(time[1]) if len(time) >= 2 else 0)
        # print( data['time'], time)
    elif 'мин' in minutes:
        data['time'] = int(minutes[:-len(' мин')])
    else:
        data['proteins fats carbohydrates'] = minutes
        data['time'] = 0

    return data

def swap_calories(data):
    c = data['calories']

    if '/' in c:
        data['calories'] = float(c[:-len(' ккал/порция')])
    else:
        data['calories'] = 0
        data['number_of_servings'] = c if not 'ч' in c and not 'мин' in c else 0

    return data

def divide_columns(data):
    k = data['proteins fats carbohydrates']

    if len(k) != 0 and not 'мин' in k and not 'ч' in k:
        x = list(map(float, k.split('г/')))
        # print(x)
        data['proteins'] = x[0]
        data['fats'] = x[1]
        data['carbohydrates'] = x[2]

    return data
df = df.apply(swap_calories, axis=1)
df = df.apply(to_min, axis=1)
# df = df.apply(swap_calories, axis=1)
df = df.apply(divide_columns, axis=1)

df['number_of_servings'] = df['number_of_servings'].map(lambda x: x if x != '' else 0)
df = df.astype({'rating':np.float32, 'number_of_servings': np.int32})

In [20]:
import locale
locale.setlocale(locale.LC_TIME, 'ru_RU.UTF-8')
df['date'] = pd.to_datetime(df['date'], format='%d %B %Y г.')

In [21]:
np.unique(df['number_of_servings'])

array([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,   10,
         11,   12,   13,   14,   15,   16,   18,   20,   21,   22,   24,
         25,   30,   32,   34,   40,   45,   50,   64,   80,  850, 1300],
      dtype=int32)

In [9]:
'58 имн'[:-len(' мин')], ' 2 ч 30'.strip().replace('ч', '').split(), '40.01 г/42.98 г/23.81'.split('г/'), map(float,[1, 2])

('58', ['2'], ['40.01 ', '42.98 ', '23.81'], <map at 0x7f475b326cb0>)

In [22]:
df.info() # Превести колонки в int и разделить некоторые значения/ привести часы в минуты

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10505 entries, 0 to 10504
Data columns (total 15 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   calories                     10505 non-null  float64       
 1   carbohydrates                2918 non-null   float64       
 2   category                     10505 non-null  object        
 3   complexity                   10505 non-null  object        
 4   date                         10505 non-null  datetime64[ns]
 5   fats                         2918 non-null   float64       
 6   ingredients                  10505 non-null  object        
 7   number_of_servings           10505 non-null  int32         
 8   proteins                     2918 non-null   float64       
 9   proteins fats carbohydrates  10505 non-null  object        
 10  rating                       10505 non-null  float32       
 11  tags                         10505 non-nu

In [23]:
df.loc[1].text, len(df.loc[1].text)

('Котлеты из цветной капусты, которые так понравились моим гостям. Похожи на кабачковые оладьи, только ещё вкуснее.\nШаг 1Капусту залить водой и варить 5 минут после закипания. Отварную капусту и ломтик белого хлеба размоченного в молоке, измельчить с помощью блендера, затем добавить яйца, соль и муку. Перемешать.\n\n\nШаг 2Сформировать котлетки и жарить с двух сторон до золотистой корочки (Я жарила на огне чуть выше среднего по 1 минуте с каждой стороны).\n\n\nШаг 3Выложить капустные котлеты на бумажные полотенца и промокнуть лишнее масло. Подавать со сметаной или с любым другим соусом. Приятного аппетита!',
 605)

In [25]:
df.to_csv('cookbook.csv')

In [26]:
f =pd.read_csv('cookbook.csv')
f

,Unnamed: 0,calories,carbohydrates,category,complexity,date,fats,ingredients,number_of_servings,proteins,proteins fats carbohydrates,rating,tags,text,time,title
0,0,585.02,34.09,Пицца,легко,2023-09-11,36.17,"['основа для пиццы – 1 шт.', 'томатный соус – ...",2,30.61,30.61 г/36.17 г/34.09,5.00,"['Основное блюдо', 'Вегетарианство']","Если вы любите пиццу Пепперони, то вам обязате...",30,Пицца Мясное ассорти
1,1,0.00,NaN,Салат Цезарь,средне,2020-11-18,NaN,"['капуста цветная 1 кг', 'яйцо куриное - 2 шт....",4,NaN,NaN,0.00,"['Основное блюдо', 'Вегетарианство: Ово-лакто'...","Котлеты из цветной капусты, которые так понрав...",0,Котлеты из цветной капусты
2,2,459.22,30.20,Блюда с картошкой,легко,2015-02-16,30.66,"['1 кг картофеля для пюре', '250–300 г тонких ...",4,15.38,15.38 г/30.66 г/30.20,5.00,"['Жарение', 'Обед', 'Ужин', 'Основное блюдо', ...","Шаг 1\n\n\nОчистите картофель, разрежьте кажду...",45,Картофельное пюре с чесноком и беконом
3,3,266.92,NaN,Блюда с картошкой,средне,2017-12-11,NaN,"['мука пшеничная 600 г', 'дрожжи быстродейству...",15,NaN,1 ч,5.00,"['Хлебопечка', 'Выпекание', 'Жарение', 'Выпечк...","Румяные, мягкие домашние пирожки с картофелем ...",135,Пирожки с картофелем и луком
4,4,0.00,NaN,Блины и блинчики,легко,2011-09-30,NaN,"['семга слабосоленая – 150 г', 'сметана – 175-...",1,NaN,NaN,0.00,"['Основное блюдо', 'Вегетарианство']","Шаг 1Взбить миксером яйца с сахаром, солью и м...",0,Блины закусочные Традиционные
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10500,10500,0.00,NaN,Котлеты,средне,2014-03-23,NaN,['фасоль «Черный глаз» белая пестрая ТМ Мист...,1,NaN,NaN,0.00,"['Основное блюдо', 'Вегетарианство: Строго', '...","Шаг 1\n\n\nПриготовить ""фарш"" для котлет. Зали...",0,Гречневые котлеты с фасолью и соусом из авокадо
10501,10501,0.00,NaN,Праздничные закуски,средне,2014-12-14,NaN,"['куриная печень - 500 г', 'лук репчатый - 1 ш...",12,NaN,NaN,4.33,"['Европейская кухня', 'Завтрак', 'Закуска', 'В...",Очень вкусный и нежный паштет из куриной печен...,15,Паштет из куриной печени с коньяком и гранатов...
10502,10502,183.38,22.41,Тушеные овощи,легко,2007-06-22,8.97,"['2 стебля черешкового сельдерея', '100 г зеле...",4,2.82,2.82 г/8.97 г/22.41,4.50,"['Итальянская кухня', 'Закуска', 'Вегетарианст...","Капоната – сицилийское блюдо из баклажанов, по...",30,Капоната
10503,10503,948.00,73.00,Пироги,средне,2008-01-16,56.00,"['мука ржаная – 100 г', 'масло сливочное – 250...",6,40.00,40.00 г/56.00 г/73.00,0.00,['Выпечка'],"Шаг 1Пшеничную и ржаную муку просеять в миску,...",0,Сочень архангельский с рыбой


In [27]:
f.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10505 entries, 0 to 10504
Data columns (total 16 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Unnamed: 0                   10505 non-null  int64  
 1   calories                     10505 non-null  float64
 2   carbohydrates                2918 non-null   float64
 3   category                     10505 non-null  object 
 4   complexity                   10332 non-null  object 
 5   date                         10505 non-null  object 
 6   fats                         2918 non-null   float64
 7   ingredients                  10505 non-null  object 
 8   number_of_servings           10505 non-null  int64  
 9   proteins                     2918 non-null   float64
 10  proteins fats carbohydrates  4057 non-null   object 
 11  rating                       10505 non-null  float64
 12  tags                         10505 non-null  object 
 13  text            